# Setup

In [1]:
import pandas as pd
import dataclasses
import nba_api
import numpy as np
from functools import reduce

## nba_api packages

In [2]:
from nba_api.stats.static.teams import get_teams, find_teams_by_nickname
#from nba_api.stats.static.players import 

from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import BoxScoreTraditionalV2
from nba_api.stats.endpoints import BoxScorePlayerTrackV2
from nba_api.stats.endpoints import BoxScoreScoringV2
from nba_api.stats.endpoints import BoxScoreMiscV2

# Define functions

In [4]:
def combine_shooting_fields(df, makes_col, att_col, field_name):
    df[field_name] = df[makes_col].astype(int).astype(str) + '-' + df[att_col].astype(int).astype(str)

In [5]:
def format_names(name):
    split_names = name.split(' ')
    first_name = split_names[0][0] + '.'
    
    split_names[0] = first_name
    
    return ' '.join(split_names)

In [6]:
# format percentages
def format_pct(pct):
    pct = round(pct*100, 2)
    out_format = str(pct) + '%'
    return out_format

In [7]:
# mark home or away game

def assign_home(matchup):
    #split_matchup = matchup.split(' ')
    
    if matchup[1] == '@':
        return np.array(['away', 'home'])
    else:
        return np.array(['home', 'away'])

# Get Spurs' box scores

In [3]:
# Get Spurs' team info
all_teams = get_teams()
spurs_info = find_teams_by_nickname('spurs') # find spurs nickname
spurs_id = spurs_info[0]['id'] # get spurs team ID

In [8]:
# find all Spurs games
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=spurs_id,
                                              season_nullable='2019-20')
spurs_games_df = gamefinder.get_data_frames()[0]
spurs_games_df.head(n=10)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22019,1610612759,SAS,San Antonio Spurs,0021901314,2020-08-13,SAS @ UTA,L,239,112,...,0.667,9,35,44,24,4,8,15,20,-6.0
1,22019,1610612759,SAS,San Antonio Spurs,0021901298,2020-08-11,SAS vs. HOU,W,240,123,...,0.900,9,50,59,28,10,5,22,20,18.0
2,22019,1610612759,SAS,San Antonio Spurs,0021901287,2020-08-09,SAS @ NOP,W,241,122,...,0.969,15,36,51,22,12,6,17,27,9.0
3,22019,1610612759,SAS,San Antonio Spurs,0021901274,2020-08-07,SAS vs. UTA,W,240,119,...,0.769,8,39,47,27,10,5,13,18,8.0
4,22019,1610612759,SAS,San Antonio Spurs,0021901264,2020-08-05,SAS vs. DEN,L,239,126,...,0.852,10,25,35,29,10,5,7,20,-6.0
5,22019,1610612759,SAS,San Antonio Spurs,0021901254,2020-08-03,SAS @ PHI,L,240,130,...,0.897,13,31,44,21,7,6,14,23,-2.0
6,22019,1610612759,SAS,San Antonio Spurs,0021901246,2020-08-02,SAS @ MEM,W,241,108,...,0.727,12,37,49,22,5,6,16,20,2.0
7,22019,1610612759,SAS,San Antonio Spurs,0021901237,2020-07-31,SAS vs. SAC,W,240,129,...,0.815,9,33,42,32,6,4,15,17,9.0
8,12019,1610612759,SAS,San Antonio Spurs,0011900130,2020-07-28,SAS @ IND,W,241,118,...,0.905,7,41,48,29,8,8,12,18,7.0
9,12019,1610612759,SAS,San Antonio Spurs,0011900115,2020-07-25,SAS vs. BKN,L,240,119,...,0.857,12,45,57,21,4,11,24,32,-5.0


In [9]:
spurs_games = list(spurs_games_df.GAME_ID.unique()) # grab unique list of game IDs

# Get Player Box Scores

In [10]:
player_box_scores_df = BoxScoreTraditionalV2(game_id=spurs_games[0]).data_sets[0].get_data_frame()
player_box_scores_df.fillna(0, inplace=True)
player_box_scores_df.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0021901314,1610612759,SAS,San Antonio,1629640,Keldon Johnson,F,,30:11,8.0,...,0.0,2.0,2.0,0.0,1.0,0.0,3.0,4.0,24.0,4.0
1,0021901314,1610612759,SAS,San Antonio,1629677,Luka Samanic,F,,31:27,5.0,...,1.0,5.0,6.0,3.0,0.0,1.0,1.0,4.0,16.0,7.0
2,0021901314,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,C,,21:08,5.0,...,2.0,3.0,5.0,3.0,1.0,2.0,1.0,3.0,10.0,-1.0
3,0021901314,1610612759,SAS,San Antonio,1629022,Lonnie Walker IV,G,,34:04,6.0,...,1.0,2.0,3.0,4.0,1.0,0.0,1.0,0.0,15.0,-12.0
4,0021901314,1610612759,SAS,San Antonio,1627749,Dejounte Murray,G,,36:16,5.0,...,1.0,13.0,14.0,7.0,0.0,0.0,6.0,3.0,12.0,4.0


In [11]:
player_box_scores_df.dtypes

GAME_ID               object
TEAM_ID                int64
TEAM_ABBREVIATION     object
TEAM_CITY             object
PLAYER_ID              int64
PLAYER_NAME           object
START_POSITION        object
COMMENT               object
MIN                   object
FGM                  float64
FGA                  float64
FG_PCT               float64
FG3M                 float64
FG3A                 float64
FG3_PCT              float64
FTM                  float64
FTA                  float64
FT_PCT               float64
OREB                 float64
DREB                 float64
REB                  float64
AST                  float64
STL                  float64
BLK                  float64
TO                   float64
PF                   float64
PTS                  float64
PLUS_MINUS           float64
dtype: object

In [12]:
player_box_scores_df.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'START_POSITION', 'COMMENT', 'MIN', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS'],
      dtype='object')

In [13]:
player_box_scores_df[['FG_PCT', 'FG3_PCT', 'FT_PCT']].head()

,FG_PCT,FG3_PCT,FT_PCT
0,0.667,0.333,0.875
1,0.500,0.600,0.750
2,0.833,0.000,0.000
3,0.429,1.000,0.000
4,0.357,0.000,1.000


In [14]:
# Change some field floats to integers
conv_list = list(player_box_scores_df.columns)[9:]

for col in conv_list:
    if 'PCT' in col:
        pass
    else:
        player_box_scores_df[col] = player_box_scores_df[col].astype(int)

In [15]:
player_box_scores_df.dtypes

GAME_ID               object
TEAM_ID                int64
TEAM_ABBREVIATION     object
TEAM_CITY             object
PLAYER_ID              int64
PLAYER_NAME           object
START_POSITION        object
COMMENT               object
MIN                   object
FGM                    int64
FGA                    int64
FG_PCT               float64
FG3M                   int64
FG3A                   int64
FG3_PCT              float64
FTM                    int64
FTA                    int64
FT_PCT               float64
OREB                   int64
DREB                   int64
REB                    int64
AST                    int64
STL                    int64
BLK                    int64
TO                     int64
PF                     int64
PTS                    int64
PLUS_MINUS             int64
dtype: object

In [16]:
combine_shooting_fields(player_box_scores_df, 'FGM', 'FGA', 'FG')
combine_shooting_fields(player_box_scores_df, 'FG3M', 'FG3A', '3P')
combine_shooting_fields(player_box_scores_df, 'FTM', 'FTA', 'FT')

In [17]:
# Convert percentages format to XX.X
#player_box_scores_df['FG_PCT'] = player_box_scores_df['FG_PCT'].apply(format_pct)
#player_box_scores_df['FG3_PCT'] = player_box_scores_df['FG3_PCT'].apply(format_pct)
#player_box_scores_df['FT_PCT'] = player_box_scores_df['FT_PCT'].apply(format_pct)

In [18]:
# apply function to format names
player_box_scores_df['name'] = player_box_scores_df.PLAYER_NAME.apply(format_names)

In [19]:
player_box_scores_df.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'START_POSITION', 'COMMENT', 'MIN', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS',
       'FG', '3P', 'FT', 'name'],
      dtype='object')

In [20]:
# This dataframe stores the players box scores for that game
players_box_outdf = player_box_scores_df[['name', 'TEAM_ABBREVIATION', 'MIN', 'PTS', 'FG', '3P', 'FT', \
                                          'AST', 'REB','DREB', 'OREB', 'BLK', 'STL', 'TO', 'PF', 'COMMENT']].copy()

In [21]:
spurs_outdf = players_box_outdf[players_box_outdf.TEAM_ABBREVIATION == 'SAS']
spurs_outdf

,name,TEAM_ABBREVIATION,MIN,PTS,FG,3P,FT,AST,REB,DREB,OREB,BLK,STL,TO,PF,COMMENT
0,K. Johnson,SAS,30:11,24,8-12,1-3,7-8,0,2,2,0,0,1,3,4,
1,L. Samanic,SAS,31:27,16,5-10,3-5,3-4,3,6,5,1,1,0,1,4,
2,J. Poeltl,SAS,21:08,10,5-6,0-0,0-5,3,5,3,2,2,1,1,3,
3,L. Walker IV,SAS,34:04,15,6-14,3-3,0-0,4,3,2,1,0,1,1,0,
4,D. Murray,SAS,36:16,12,5-14,0-2,2-2,7,14,13,1,0,0,6,3,
5,M. Belinelli,SAS,29:43,16,6-11,3-6,1-2,0,0,0,0,0,0,0,2,
6,Q. Weatherspoon,SAS,3:17,1,0-0,0-0,1-2,0,1,1,0,0,0,0,1,
7,D. Eubanks,SAS,27:14,8,1-6,0-0,6-8,3,7,5,2,4,0,1,1,
8,C. Metu,SAS,26:40,10,3-6,0-1,4-5,4,6,4,2,1,1,2,2,
9,D. DeRozan,SAS,0,0,0-0,0-0,0-0,0,0,0,0,0,0,0,0,DNP - Coach's Decision


In [22]:
opp_outdf = players_box_outdf[players_box_outdf.TEAM_ABBREVIATION != 'SAS']
opp_outdf

,name,TEAM_ABBREVIATION,MIN,PTS,FG,3P,FT,AST,REB,DREB,OREB,BLK,STL,TO,PF,COMMENT
13,J. Ingles,UTA,12:20,3,1-4,1-3,0-0,3,1,1,0,0,1,0,0,
14,R. O'Neale,UTA,11:20,2,1-4,0-2,0-0,0,1,1,0,2,0,0,1,
15,T. Bradley,UTA,21:41,6,3-7,0-0,0-0,1,10,7,3,1,0,1,1,
16,J. Clarkson,UTA,14:02,11,4-12,3-9,0-0,1,2,2,0,0,0,2,1,
17,D. Mitchell,UTA,11:20,11,3-5,3-4,2-2,3,4,4,0,0,0,0,1,
18,G. Niang,UTA,22:00,13,5-10,3-8,0-0,2,5,4,1,0,0,1,1,
19,M. Oni,UTA,25:49,10,4-8,2-5,0-0,2,2,1,1,0,1,1,3,
20,E. Davis,UTA,7:29,11,5-6,0-0,1-1,1,6,2,4,0,0,0,2,
21,J. Morgan,UTA,21:49,6,3-4,0-0,0-0,1,3,1,2,1,1,1,5,
22,J. Brantley,UTA,28:59,13,6-13,0-3,1-2,6,4,3,1,1,1,0,2,


In [23]:
test_dict = opp_outdf.to_dict(orient = 'records')

import json

with open('test.json', 'w') as outfile:
    json.dump(test_dict, outfile)

In [24]:
# output all data files
#players_box_outdf.to_json('data/players_full_box_score.json', orient='index')
spurs_outdf.to_json('data/players_spurs_box_score.json', orient = 'records')
opp_outdf.to_json('data/players_opp_box_score.json', orient = 'records')

# Get team box scores

In [25]:
team_box_scores = BoxScoreTraditionalV2(game_id=spurs_games[0]).data_sets
team_box_score_df = team_box_scores[1].get_data_frame()

In [26]:
team_box_scores[2].get_data_frame()

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,STARTERS_BENCH,MIN,FGM,FGA,FG_PCT,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS
0,0021901314,1610612762,Jazz,UTA,Utah,Starters,70:43,12,32,0.375,...,1.000,3,15,18,8,1,3,3,4,33
1,0021901314,1610612762,Jazz,UTA,Utah,Bench,169:17,35,69,0.507,...,0.875,9,16,25,17,6,2,5,19,85
2,0021901314,1610612759,Spurs,SAS,San Antonio,Starters,153:06,29,56,0.518,...,0.632,5,25,30,17,3,3,12,14,77
3,0021901314,1610612759,Spurs,SAS,San Antonio,Bench,83:37,10,23,0.435,...,0.733,4,9,13,7,1,5,3,5,34


In [27]:
team_box_score_df.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CITY',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS',
       'PLUS_MINUS'],
      dtype='object')

In [28]:
game_matchup = spurs_games_df[spurs_games_df['GAME_ID'] == spurs_games[0]].MATCHUP.str.split(' ')

opponent_abbr = game_matchup.iloc[0][2]

home_or_away = assign_home(game_matchup.iloc[0])

In [29]:
home_dict = {
    'TEAM_ABBREVIATION': ['SAS', opponent_abbr],
    'home': home_or_away
}

temp_home_df = pd.DataFrame(data=home_dict)
temp_home_df

,TEAM_ABBREVIATION,home
0,SAS,away
1,UTA,home


In [30]:
temp_home_df.dtypes

TEAM_ABBREVIATION    object
home                 object
dtype: object

In [31]:
team_box_score_df.dtypes

GAME_ID               object
TEAM_ID                int64
TEAM_NAME             object
TEAM_ABBREVIATION     object
TEAM_CITY             object
MIN                   object
FGM                    int64
FGA                    int64
FG_PCT               float64
FG3M                   int64
FG3A                   int64
FG3_PCT              float64
FTM                    int64
FTA                    int64
FT_PCT               float64
OREB                   int64
DREB                   int64
REB                    int64
AST                    int64
STL                    int64
BLK                    int64
TO                     int64
PF                     int64
PTS                    int64
PLUS_MINUS           float64
dtype: object

In [32]:
team_box_score_df['2PM'] = team_box_score_df['FGM'] - team_box_score_df['FG3M']
team_box_score_df['2PA'] = team_box_score_df['FGA'] - team_box_score_df['FG3A']
team_box_score_df['2P_PCT'] = round((team_box_score_df['2PM'] / team_box_score_df['2PA']), 3)

In [33]:
team_box_score_df

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,FGM,FGA,FG_PCT,FG3M,...,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,2PM,2PA,2P_PCT
0,0021901314,1610612762,Jazz,UTA,Utah,240:00,47,101,0.465,15,...,25,7,5,8,23,118,6.0,32,55,0.582
1,0021901314,1610612759,Spurs,SAS,San Antonio,240:00,39,79,0.494,10,...,24,4,8,15,20,112,-6.0,29,59,0.492


In [34]:
# Reassign format to 7-9 FGA-FGM
combine_shooting_fields(team_box_score_df, 'FGM', 'FGA', 'FG')
combine_shooting_fields(team_box_score_df, 'FG3M', 'FG3A', '3P')
combine_shooting_fields(team_box_score_df, 'FTM', 'FTA', 'FT')
combine_shooting_fields(team_box_score_df, '2PM', '2PA', '2P')

In [35]:
# Convert percentages format to XX.X
team_box_score_df['2P_PCT'] = team_box_score_df['2P_PCT'].apply(format_pct)
team_box_score_df['FG3_PCT'] = team_box_score_df['FG3_PCT'].apply(format_pct)
team_box_score_df['FG_PCT'] = team_box_score_df['FG_PCT'].apply(format_pct)
team_box_score_df['FT_PCT'] = team_box_score_df['FT_PCT'].apply(format_pct)

# append home or away column
team_box_score_df = team_box_score_df.join(temp_home_df.set_index('TEAM_ABBREVIATION'), on='TEAM_ABBREVIATION')

In [36]:
team_box_score_df

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,FGM,FGA,FG_PCT,FG3M,...,PTS,PLUS_MINUS,2PM,2PA,2P_PCT,FG,3P,FT,2P,home
0,0021901314,1610612762,Jazz,UTA,Utah,240:00,47,101,46.5%,15,...,118,6.0,32,55,58.2%,47-101,15-46,9-10,32-55,home
1,0021901314,1610612759,Spurs,SAS,San Antonio,240:00,39,79,49.4%,10,...,112,-6.0,29,59,49.2%,39-79,10-20,24-36,29-59,away


In [37]:
full_teams_df = team_box_score_df[['TEAM_ABBREVIATION', 'home', 'PTS', 'FG', 'FG_PCT', '3P', 'FG3_PCT', \
                                  'FT', 'FT_PCT', 'AST', 'REB', 'DREB', 'OREB', 'BLK', \
                                  'STL', 'TO', 'PF']].copy()

In [38]:
spurs_team_box_df = full_teams_df[full_teams_df['TEAM_ABBREVIATION'] == 'SAS']
opp_team_box_df = full_teams_df[full_teams_df['TEAM_ABBREVIATION'] != 'SAS']

In [39]:
# output team stats df

#full_teams_df.to_json('data/team_box_scores_full.json', orient='records')
spurs_team_box_df.to_json('data/team_box_scores_spurs.json', orient='records')
opp_team_box_df.to_json('data/team_box_scores_opp.json', orient = 'records')

# Get extra team box score measures

In [40]:
misc_stats_df = BoxScoreMiscV2(game_id=spurs_games[0]).data_sets[1].get_data_frame()
misc_stats_df.head()

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,PTS_OFF_TOV,PTS_2ND_CHANCE,PTS_FB,PTS_PAINT,OPP_PTS_OFF_TOV,OPP_PTS_2ND_CHANCE,OPP_PTS_FB,OPP_PTS_PAINT,BLK,BLKA,PF,PFD
0,0021901314,1610612762,Jazz,UTA,Utah,240:00,20.0,21.0,9.0,62.0,14.0,9.0,9.0,44.0,5,8,23,20
1,0021901314,1610612759,Spurs,SAS,San Antonio,240:00,14.0,9.0,9.0,44.0,20.0,21.0,9.0,62.0,8,5,20,23


In [41]:
# misc stats
misc_stats_df = BoxScoreMiscV2(game_id=spurs_games[0]).data_sets[1].get_data_frame()

# isolate bench points
bench_stats_df = team_box_scores[2].get_data_frame()


In [42]:
misc_stats_df.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CITY',
       'MIN', 'PTS_OFF_TOV', 'PTS_2ND_CHANCE', 'PTS_FB', 'PTS_PAINT',
       'OPP_PTS_OFF_TOV', 'OPP_PTS_2ND_CHANCE', 'OPP_PTS_FB', 'OPP_PTS_PAINT',
       'BLK', 'BLKA', 'PF', 'PFD'],
      dtype='object')

In [43]:
# keep stats: 
# tot ast, tot reb, pts in paint, fb pts, 2nd chance pts, pts off tov, bench pts

team_box_score_df.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CITY',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS',
       'PLUS_MINUS', '2PM', '2PA', '2P_PCT', 'FG', '3P', 'FT', '2P', 'home'],
      dtype='object')

In [44]:
# get assists and rebounds
team_bs_df = team_box_score_df[['GAME_ID', 'TEAM_ABBREVIATION', 'AST', 'REB']]
team_bs_df

,GAME_ID,TEAM_ABBREVIATION,AST,REB
0,0021901314,UTA,25,43
1,0021901314,SAS,24,44


In [45]:
# get bench points
bench_points_df = bench_stats_df[bench_stats_df['STARTERS_BENCH'] == 'Bench'][['GAME_ID', 
                                                                               'TEAM_ABBREVIATION', 'PTS']]
bench_points_df

,GAME_ID,TEAM_ABBREVIATION,PTS
1,0021901314,UTA,85
3,0021901314,SAS,34


In [46]:
# pts in paint, fb points, 2nd chance pts, pts off tov

final_stats_df = misc_stats_df[['GAME_ID', 'TEAM_ABBREVIATION', 'PTS_PAINT', 'PTS_2ND_CHANCE',
                               'PTS_FB', 'PTS_OFF_TOV']]

final_stats_df

,GAME_ID,TEAM_ABBREVIATION,PTS_PAINT,PTS_2ND_CHANCE,PTS_FB,PTS_OFF_TOV
0,0021901314,UTA,62.0,21.0,9.0,20.0
1,0021901314,SAS,44.0,9.0,9.0,14.0


In [47]:
data_frames = [team_bs_df, final_stats_df, bench_points_df]
misc_bs_df = reduce(lambda left, right: pd.merge(left, right, on=['GAME_ID', 'TEAM_ABBREVIATION'],
                                               how = 'left'), data_frames)

In [48]:
# rename pts to bench points

misc_bs_df.rename(columns={'PTS': 'bench_pts'}, inplace=True)

In [49]:
for col in list(misc_bs_df.columns[2:]):
    misc_bs_df[col] = misc_bs_df[col].astype(int)

In [50]:
misc_bs_df

,GAME_ID,TEAM_ABBREVIATION,AST,REB,PTS_PAINT,PTS_2ND_CHANCE,PTS_FB,PTS_OFF_TOV,bench_pts
0,0021901314,UTA,25,43,62,21,9,20,85
1,0021901314,SAS,24,44,44,9,9,14,34


In [51]:
# export misc stats
misc_bs_df.to_json('data/misc_box_stats.json', orient='records')

# Gather shooting efficiency stats